In [31]:
import pandas as pd
import clean_job_title as c
import requests
import json
import time

suggest_word_dict = pd.read_csv("words_not_in_en_2.csv")

In [32]:
err_title_pd = pd.read_csv("error_title.txt")

In [33]:
def clean_job_title(job_title):
    clean_title, clean_score = c.standard_job_title(job_title, suggest_word_dict)
    return clean_title

In [34]:
err_title_pd['clean_title'] = err_title_pd.apply(lambda x: clean_job_title(x['job_title']), axis=1)

In [35]:
err_title_pd.head()

,job_title,result_code,clean_title
0,SESpecialist,404,sespecialist
1,photeditor,404,photeditor
2,Production Arist,404,production arist
3,photArist,404,photarist
4,User Interface (UI) Developer,404,user interface developer


In [36]:
err_title_pd['result'] = ''
err_title_pd['error_message'] = ''

In [40]:
url = 'https://goodwin.api.relinklabs.com/v1/explore?title='

for index, row in err_title_pd.iterrows():
    
    try:
        title = row['clean_title']

        url_title = url + title.replace(' ','%20')
        r = requests.get(url_title)

        err_title_pd.set_value(index, 'result_code', r.status_code)

        print("Index: ", index)
        print("r.json(): ", r.json())

        if(r.status_code == 200):
            err_title_pd.set_value(index, 'result', str(r.json()))
        else:
            err_title_pd.set_value(index, 'error_message', str(r.json()))
    except Exception as e:
        print ("Error: ", e)
        time.sleep(5)
        continue
        
        

Index:  0
r.json():  {'errorType': 'Exception', 'errorMessage': 'NotFoundError: Sorry we could not find any information for title: sespecialist', 'stackTrace': [['/var/task/function.py', 80, 'lambda_handler', "raise Exception('NotFoundError: Sorry we could not find any information for title: ' + title or '')"]]}
Index:  1
r.json():  {'errorType': 'Exception', 'errorMessage': 'NotFoundError: Sorry we could not find any information for title: photeditor', 'stackTrace': [['/var/task/function.py', 80, 'lambda_handler', "raise Exception('NotFoundError: Sorry we could not find any information for title: ' + title or '')"]]}
Index:  2
r.json():  {'errorType': 'Exception', 'errorMessage': 'NotFoundError: Sorry we could not find any information for title: production arist', 'stackTrace': [['/var/task/function.py', 80, 'lambda_handler', "raise Exception('NotFoundError: Sorry we could not find any information for title: ' + title or '')"]]}
Index:  3
r.json():  {'errorType': 'Exception', 'errorMe

KeyboardInterrupt: 

In [43]:
err_title_pd[err_title_pd.result_code == 200]

,job_title,result_code,clean_title,result,error_message
4,User Interface (UI) Developer,200,user interface developer,"{'stemmedJob': 'user interfac develop', 'educa...",
8,Economics (quantitative),200,economics,"{'stemmedJob': 'econom', 'education': [{'score...",
10,Psychometrician (measuring mental capacities),200,psychometrician,"{'stemmedJob': 'psychometrician', 'education':...",
